In [10]:
import requests as rq
from bs4 import BeautifulSoup as bs
from bs4.element import Comment
import codecs
import collections
import pandas as pd
import numpy as np
import scipy as scp
import re
import os
from unicodedata import normalize

In [4]:
def remover_acentos_e_pontuacao(txt):
    sem_acentos = normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')
    return re.sub('[^a-zA-Z0-9 ]', '', sem_acentos)
print (remover_acentos_e_pontuacao("áàãâä! éèêë? íì&#297;îï, óòõôö; úù&#361;ûü"))


aaaaa eeee ii297ii ooooo uu361uu


In [5]:
# função utilizada para baixar uma página html e savar em name
def getPage(url, name, codec):
    
    if not os.path.isfile(name):
        page = rq.get(url)
        f = codecs.open(name, 'w', codec)
        f.write(page.text)
        f.close

In [6]:
# utilizado para filtrar apenas of elementos html que são visiveis para o usuário
def tag_visible(element):
    
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

# utilizado para pegar apenas o texto da página html
def getText(arquivo):        
    bsFile = bs(open(arquivo),"html.parser")
    texts = bsFile.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    words = [t.strip() for t in visible_texts]
    words = filter(lambda x: len(x)>0, words)
    words = map(lambda x: x.lower(), words)
    text = " ".join(words)
    return remover_acentos_e_pontuacao(text)

# utilizado para pegar as palavras que aparecem em uma página html (apenas páginas salvas localmente)
def getUniqueWords(arquivo):
    text = getText(arquivo)
    words = text.split(" ")
    uniqueWords = collections.Counter(words)
    return [word for word in uniqueWords.keys()]

def getAllUniqueWords(diretorio):
    files = os.listdir(diretorio)
    files = filter(lambda x: x[-5:].lower() == ".html", files)
    allWords = []
    for index, f in enumerate(files):
        allWords = allWords + getUniqueWords(diretorio + "/" + f)
        print (index, len(allWords))
        
    uniqueWords = collections.Counter(allWords)
    return [word for word in uniqueWords.keys()]

In [18]:
# print (getUniqueWords("sites/0www.riachuelo.com.br.html"))
fr = codecs.open("palavrasacento.txt", "w", "utf-8")
files = os.listdir("sites")
files = filter(lambda x: x[-5:].lower() == ".html", files)
words = []
for index, f in enumerate(files):
    w = getUniqueWords("sites/" + f)
    w = filter(lambda word: len(word) > 0, w)
    words += w
    print ("está no arquivo: " + str(f))
    
uwords = []
for w in words:
    if w not in uwords:
        uwords.append(w)
        
fr.write(",".join(uwords) + "\n")
fr.close()
    

está no arquivo: 31www.lojasrenner.com.br.html
está no arquivo: 148www.gregory.com.br.html
está no arquivo: 142www.gregory.com.br.html
está no arquivo: 137www.gregory.com.br.html
está no arquivo: 209www.dafiti.com.br.html
está no arquivo: 164www.zattini.com.br.html
está no arquivo: 27www.lojasrenner.com.br.html
está no arquivo: 155www.zattini.com.br.html
está no arquivo: 116www.cea.com.br.html
está no arquivo: 90www.marisa.com.br.html
está no arquivo: 102www.marisa.com.br.html
está no arquivo: 67mariadonata.com.br.html
está no arquivo: 42www.damyller.com.br.html
está no arquivo: 194produto.mercadolivre.com.br.html
está no arquivo: 201www.dafiti.com.br.html
está no arquivo: 208dafitimag.dafiti.com.br.html
está no arquivo: 169www.zattini.com.br.html
está no arquivo: 51www.damyller.com.br.html
está no arquivo: 145www.gregory.com.br.html
está no arquivo: 130www.gregory.com.br.html
está no arquivo: 126www.cea.com.br.html
está no arquivo: 163www.zattini.com.br.html
está no arquivo: 217www.da

está no arquivo: 86pesquisa.marisa.com.br.html
está no arquivo: 172www.zattini.com.br.html
está no arquivo: 204www.dafiti.com.br.html
está no arquivo: 114www.cea.com.br.html
está no arquivo: 119www.cea.com.br.html
está no arquivo: 71mariadonata.com.br.html
está no arquivo: 53www.damyller.com.br.html
está no arquivo: 179loja.mercadolivre.com.br.html
está no arquivo: 198produto.mercadolivre.com.br.html
está no arquivo: 171www.zattini.com.br.html
está no arquivo: 97www.marisa.com.br.html
está no arquivo: 105www.marisa.com.br.html
está no arquivo: 117www.cea.com.br.html
está no arquivo: 82mariadonata.com.br.html
está no arquivo: 188roupas.mercadolivre.com.br.html
está no arquivo: 222www.dafiti.com.br.html
está no arquivo: 44www.damyller.com.br.html
está no arquivo: 18www.riachuelo.com.br.html
está no arquivo: 72mariadonata.com.br.html
está no arquivo: 178www.mercadolivre.com.br.html
está no arquivo: 78mariadonata.com.br.html
está no arquivo: 218www.dafiti.com.br.html
está no arquivo: 150ww

In [12]:
fr = codecs.open("palavrasacento.txt", "r", "utf-8")
uwordsSelected = fr.read().split(",")
fr.close()

In [8]:
# utilizado para ler o arquivo com as páginas utilizadas no projeto
data_set = pd.read_csv("sites.csv")

In [94]:
# utizado para baixar as páginas html que foram utilizadas no projeto

padrao = '((www\.)?[a-zA-Z0-9]+\.[a-zA-Z0-9]+(\.[a-zA-Z0-9]+)*)'
for i, site in enumerate(data_set.url):
    name = str(i) + re.findall(padrao, site)[0][0] + ".html"
    getPage(url=site, name="sites/" + name, codec="utf-8")
    print "finalizou " + name

In [13]:
fr = codecs.open("baseAcentoTodos.csv", "w", "utf-8")
fr.write(",".join(uwordsSelected) + ",classePagina\n")
files = os.listdir("sites")
files = filter(lambda x: x[-5:].lower() == ".html", files)
for index, f in enumerate(files):
# if True:
    allWords = getUniqueWords("sites" + "/" + f)
    allWords = filter(lambda word: len(word) > 0, allWords) # filtrar por palavras não vazias

    palavras = collections.Counter(allWords) # contar quantidade de cada palavras

    inputSite = []
    for pp in uwordsSelected:
        inputSite.append(palavras[pp])
    
    inputSite = ",".join(map(str, inputSite))
    
    padrao = '([0-9]+)'
#     print ("finalizou " + str(re.findall(padrao, name)[0]))
    fr.write(inputSite + "," + data_set.anuncio[int(re.findall(padrao, f)[0])] + "\n")
    print ("está no arquivo: " + str(f) + " --> " + data_set.anuncio[int(re.findall(padrao, f)[0])])

fr.close()

está no arquivo: 114www.cea.com.br.html --> não
está no arquivo: 1www.riachuelo.com.br.html --> não
está no arquivo: 77mariadonata.com.br.html --> sim
está no arquivo: 95www.marisa.com.br.html --> não
está no arquivo: 178www.mercadolivre.com.br.html --> não
está no arquivo: 14www.riachuelo.com.br.html --> sim
está no arquivo: 208dafitimag.dafiti.com.br.html --> não
está no arquivo: 33www.lojasrenner.com.br.html --> sim
está no arquivo: 28www.lojasrenner.com.br.html --> não
está no arquivo: 8www.riachuelo.com.br.html --> não
está no arquivo: 204www.dafiti.com.br.html --> não
está no arquivo: 160www.zattini.com.br.html --> não
está no arquivo: 48www.damyller.com.br.html --> não
está no arquivo: 93www.marisa.com.br.html --> não
está no arquivo: 105www.marisa.com.br.html --> sim
está no arquivo: 103www.marisa.com.br.html --> sim
está no arquivo: 20www.lojasrenner.com.br.html --> não
está no arquivo: 127www.cea.com.br.html --> sim
está no arquivo: 35www.lojasrenner.com.br.html --> sim
está 

está no arquivo: 47www.damyller.com.br.html --> não
está no arquivo: 216www.dafiti.com.br.html --> sim
está no arquivo: 86pesquisa.marisa.com.br.html --> não
está no arquivo: 125www.cea.com.br.html --> sim
está no arquivo: 124www.cea.com.br.html --> sim
está no arquivo: 201www.dafiti.com.br.html --> não
está no arquivo: 69mariadonata.com.br.html --> não
está no arquivo: 167www.zattini.com.br.html --> sim
está no arquivo: 220www.dafiti.com.br.html --> sim
está no arquivo: 81mariadonata.com.br.html --> sim
está no arquivo: 183loja.mercadolivre.com.br.html --> não
está no arquivo: 122www.cea.com.br.html --> sim
está no arquivo: 117www.cea.com.br.html --> não
está no arquivo: 79mariadonata.com.br.html --> sim
está no arquivo: 102www.marisa.com.br.html --> sim
está no arquivo: 70mariadonata.com.br.html --> não
está no arquivo: 171www.zattini.com.br.html --> sim
está no arquivo: 110www.cea.com.br.html --> não
está no arquivo: 60www.damyller.com.br.html --> sim
está no arquivo: 52www.damyller

In [35]:
fr = codecs.open("teste.arff", "w", "utf-8")
for u in uwordsSelected:
    fr.write("@ATTRIBUTE " + u + " NUMERIC\n")

fr.write("@ATTRIBUTE classePagina {sim, não}\n")
fr.write("\n\n@DATA\n")
fr.close()